# Sprint 3: Advanced Relational/Conceptual design

## ToDo:

- Attempt to automate user setup etc.
- Recommended db setup (ACID)
  - Change SQL code to transactions, i.e. if db update fails (i.e. YF call) NOTHING should be executed
- Add logging to db
  - Checkpoints at each 'stage' of db, i.e.
    1. Loading YF data
    2. EF production (might need sub-stages)

# Work done

## 1. Error handling & code cleanup

To address the tasks of adding error handling and cleaning up the code with comments, let's go through each provided file and make the necessary improvements.

### `get_stock_2.py`

This file contains functions for fetching and storing stock data. We will add error handling for command-line inputs and clean up the code with comments.

```python
import yfinance as yf
from datetime import datetime, timedelta

def yf_getH(cursor, ticker, stock_id):
    """
    Fetch historical stock data for the given ticker and insert it into the database.
    
    Parameters:
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    ticker (str): Stock ticker symbol.
    stock_id (int): Unique stock identifier.
    """
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)

        for date, row in hist.iterrows():
            sql = "INSERT INTO History (HistoryID, Ticker, Date, Price) VALUES (%s, %s, %s, %s)"
            data = (stock_id, ticker, date.strftime('%Y-%m-%d'), row['Close'])
            cursor.execute(sql, data)
            print(f"Inserted history data for {ticker} on {date.strftime('%Y-%m-%d')}")
            stock_id += 1

    except Exception as e:
        print(f"Error fetching or inserting history for {ticker}: {e}")

def yf_getS(cursor, ticker, stock_id):
    """
    Fetch stock information for the given ticker and insert it into the database.
    
    Parameters:
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    ticker (str): Stock ticker symbol.
    stock_id (int): Unique stock identifier.
    """
    try:
        stock = yf.Ticker(ticker)
        stock_info = stock.info

        symbol = stock_info.get('symbol', ticker)
        sector = stock_info.get('sector', 'Unknown')
        price = stock_info.get('regularMarketPreviousClose', 0.0)
        sd = stock_info.get('beta', 0.0)
        eret = round((stock_info.get('forwardEps', 0.0) / price) * 100, 3)

        insert_statement = "INSERT INTO Stocks (StockID, Ticker, Sector, Price, SD, ERet) VALUES (%s, %s, %s, %s, %s, %s)"
        data = (stock_id, symbol, sector, price, sd, eret)
        cursor.execute(insert_statement, data)
        print(f"Inserted data for {ticker}")

    except Exception as e:
        print(f"Error fetching or inserting data for {ticker}: {e}")

def get_stock(connection, cursor, tickers):
    """
    Fetch and store stock data for a list of ticker symbols.
    
    Parameters:
    connection (MySQLConnection): MySQL connection object.
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    tickers (list): List of stock ticker symbols.
    """
    stock_id = 1

    for ticker in tickers:
        ticker = ticker.strip()
        yf_getS(cursor, ticker, stock_id)
        yf_getH(cursor, ticker, stock_id)
        stock_id += 1000

    connection.commit()
```

### `db_setup.py`

This file sets up the database schema. We will add error handling and comments.

```python
import mysql.connector
from mysql.connector import Error

def db_setup(connection, cursor):
    """
    Set up the database schema by executing SQL statements.
    
    Parameters:
    connection (MySQLConnection): MySQL connection object.
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    """
    sql_statements = [
        "DROP TABLE IF EXISTS `PortfolioHasAllocation`;",
        "DROP TABLE IF EXISTS `PortfolioHasStock`;",
        "DROP TABLE IF EXISTS `AllocationHasStock`;",
        "DROP TABLE IF EXISTS `StockHasHistory`;",
        "DROP TABLE IF EXISTS `SessionHasPortfolio`;",
        "DROP TABLE IF EXISTS `Session`;",
        "DROP TABLE IF EXISTS `Portfolio`;",
        "DROP TABLE IF EXISTS `Allocation`;",
        "DROP TABLE IF EXISTS `Stocks`;",
        "DROP TABLE IF EXISTS `History`;",
        "CREATE TABLE `Session` (`SessionID` INT PRIMARY KEY);",
        "CREATE TABLE `Portfolio` (`PortfolioID` INT PRIMARY KEY, `TotalAmt` FLOAT, `Risk` VARCHAR(64));",
        "CREATE TABLE `Allocation` (`AllocID` INT PRIMARY KEY, `Ticker` VARCHAR(10), `Amount` FLOAT);",
        "CREATE TABLE `Stocks` (`StockID` INT PRIMARY KEY, `Ticker` VARCHAR(10), `Sector` VARCHAR(64), `Price` FLOAT, `SD` FLOAT, `ERet` FLOAT);",
        "CREATE TABLE `History` (`HistoryID` INT PRIMARY KEY, `Ticker` VARCHAR(10), `Date` VARCHAR(10), `Price` FLOAT);",
        "CREATE TABLE `PortfolioHasStock` (`PortfolioID` INT, `StockID` INT, FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`), FOREIGN KEY (`StockID`) REFERENCES `Stocks`(`StockID`), PRIMARY KEY (`PortfolioID`, `StockID`));",
        "CREATE TABLE `AllocationHasStock` (`AllocID` INT, `StockID` INT, FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`), FOREIGN KEY (`StockID`) REFERENCES `Stocks`(`StockID`), PRIMARY KEY (`AllocID`, `StockID`));",
        "CREATE TABLE `StockHasHistory` (`StockID` INT, `HistoryID` INT, FOREIGN KEY (`StockID`) REFERENCES `Stocks`(`StockID`), FOREIGN KEY (`HistoryID`) REFERENCES `History`(`HistoryID`), PRIMARY KEY (`StockID`, `HistoryID`));",
        "CREATE TABLE `SessionHasPortfolio` (`SessionID` INT, `PortfolioID` INT, FOREIGN KEY (`SessionID`) REFERENCES `Session`(`SessionID`), FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`), PRIMARY KEY (`SessionID`, `PortfolioID`));",
        "CREATE TABLE `PortfolioHasAllocation` (`PortfolioID` INT, `AllocID` INT, FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`), FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`), PRIMARY KEY (`PortfolioID`, `AllocID`));"
    ]

    try:
        for sql_statement in sql_statements:
            cursor.execute(sql_statement)
        connection.commit()
        print("SQL script executed successfully.")
    except Error as e:
        print(f"Error executing SQL script: {e}")
```

### `portef.py`

This is the main script that orchestrates the database connection and data fetching. We will add error handling for command-line inputs and clean up the code with comments.

```python
#!/usr/bin/env python3

from connect import connect_to_database, close_connection
from db_setup import db_setup
from get_stock_2 import get_stock

def main():
    """
    Main function to connect to the database, set up schema, and fetch stock data.
    """
    connection, cursor = connect_to_database()
    if connection:
        try:
            tickers_input = input("Enter the stock ticker symbols (separated by commas): ").strip()
            if not tickers_input:
                raise ValueError("Ticker symbols input cannot be empty.")
            tickers = tickers_input.split(',')
            db_setup(connection, cursor)
            get_stock(connection, cursor, tickers)
        except ValueError as ve:
            print(f"Input error: {ve}")
        finally:
            close_connection(connection, cursor)

if __name__ == "__main__":
    main()
```

### `connect.py`

This file handles the database connection. We will ensure error handling and add comments.

```python
import mysql.connector
from mysql.connector import Error
import getpass

def connect_to_database():
    """
    Connect to the MySQL database based on user input.
    
    Returns:
    tuple: MySQL connection and cursor objects.
    """
    username = input("Enter your username: ").strip()
    password = getpass.getpass("Enter your password: ")
    database = input("Enter the database name: ").strip()
    change_host = input("Do you want to change the host from localhost? (y/n): ").strip().lower()

    if change_host == 'y':
        host = input("Enter the new host: ").strip() or 'localhost'
    else:
        host = 'localhost'

    try:
        connection = mysql.connector.connect(
            host=host,
            user=username,
            password=password,
            database=database
        )

        if connection.is_connected():
            print(f'Connected to MariaDB database {database} on {host}')
            cursor = connection.cursor()
            cursor.execute("SELECT VERSION()")
            db_version = cursor.fetchone()
            print(f"Database version: {db_version}")
            return connection, cursor

    except Error as e:
        print(f"Error connecting to MariaDB database: {e}")
        return None, None

def close_connection(connection, cursor):
    """
    Close the MySQL database connection.
    
    Parameters:
    connection (MySQLConnection): MySQL connection object.
    cursor (MySQLCursor): MySQL cursor object.
    """
    if cursor:
        cursor.close()
    if connection and connection.is_connected():
        connection.close()
        print('MariaDB database connection closed')
```

These changes add error handling for user inputs, clean up the code, and provide comments to explain the functionality of each part of the code. This should make the code more robust and easier to maintain.

Citations:
[1] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/c6b79763-3b0f-4a09-8ff9-6142f0343fb5/get_stock_2.py
[2] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/e62e258d-d881-483e-acf9-e1c62cde5b2a/db_setup.py
[3] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/4c11ad45-3818-40ad-b697-c3a79ad86e95/portef.py
[4] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/d2d3fb8d-41e5-4dd0-a311-e047340d036e/connect.py

## 2. Atomicity

To make the SQL code conform to ACID properties, we need to ensure that both the creation of tables and the population of tables with data are handled within transactions. This will ensure atomicity, consistency, isolation, and durability.

Here is how you can modify the provided code to use SQL transactions for both creating tables and populating them with data.

### Modifying `db_setup.py` to Use Transactions for Table Creation

First, let's modify the `db_setup.py` file to wrap the table creation statements in a transaction:

```python
import mysql.connector
from mysql.connector import Error

def db_setup(connection, cursor):
    """
    Set up the database schema by executing SQL statements within a transaction.
    
    Parameters:
    connection (MySQLConnection): MySQL connection object.
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    """
    sql_statements = [
        "DROP TABLE IF EXISTS `PortfolioHasAllocation`;",
        "DROP TABLE IF EXISTS `PortfolioHasStock`;",
        "DROP TABLE IF EXISTS `AllocationHasStock`;",
        "DROP TABLE IF EXISTS `StockHasHistory`;",
        "DROP TABLE IF EXISTS `SessionHasPortfolio`;",
        "DROP TABLE IF EXISTS `Session`;",
        "DROP TABLE IF EXISTS `Portfolio`;",
        "DROP TABLE IF EXISTS `Allocation`;",
        "DROP TABLE IF EXISTS `Stocks`;",
        "DROP TABLE IF EXISTS `History`;",
        "CREATE TABLE `Session` (`SessionID` INT PRIMARY KEY);",
        "CREATE TABLE `Portfolio` (`PortfolioID` INT PRIMARY KEY, `TotalAmt` FLOAT, `Risk` VARCHAR(64));",
        "CREATE TABLE `Allocation` (`AllocID` INT PRIMARY KEY, `Ticker` VARCHAR(10), `Amount` FLOAT);",
        "CREATE TABLE `Stocks` (`StockID` INT PRIMARY KEY, `Ticker` VARCHAR(10), `Sector` VARCHAR(64), `Price` FLOAT, `SD` FLOAT, `ERet` FLOAT);",
        "CREATE TABLE `History` (`HistoryID` INT PRIMARY KEY, `Ticker` VARCHAR(10), `Date` VARCHAR(10), `Price` FLOAT);",
        "CREATE TABLE `PortfolioHasStock` (`PortfolioID` INT, `StockID` INT, FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`), FOREIGN KEY (`StockID`) REFERENCES `Stocks`(`StockID`), PRIMARY KEY (`PortfolioID`, `StockID`));",
        "CREATE TABLE `AllocationHasStock` (`AllocID` INT, `StockID` INT, FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`), FOREIGN KEY (`StockID`) REFERENCES `Stocks`(`StockID`), PRIMARY KEY (`AllocID`, `StockID`));",
        "CREATE TABLE `StockHasHistory` (`StockID` INT, `HistoryID` INT, FOREIGN KEY (`StockID`) REFERENCES `Stocks`(`StockID`), FOREIGN KEY (`HistoryID`) REFERENCES `History`(`HistoryID`), PRIMARY KEY (`StockID`, `HistoryID`));",
        "CREATE TABLE `SessionHasPortfolio` (`SessionID` INT, `PortfolioID` INT, FOREIGN KEY (`SessionID`) REFERENCES `Session`(`SessionID`), FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`), PRIMARY KEY (`SessionID`, `PortfolioID`));",
        "CREATE TABLE `PortfolioHasAllocation` (`PortfolioID` INT, `AllocID` INT, FOREIGN KEY (`PortfolioID`) REFERENCES `Portfolio`(`PortfolioID`), FOREIGN KEY (`AllocID`) REFERENCES `Allocation`(`AllocID`), PRIMARY KEY (`PortfolioID`, `AllocID`));"
    ]

    try:
        connection.start_transaction()
        for sql_statement in sql_statements:
            cursor.execute(sql_statement)
        connection.commit()
        print("SQL script executed successfully.")
    except Error as e:
        connection.rollback()
        print(f"Error executing SQL script: {e}")
```

### Modifying `get_sh.py` to Use Transactions for Data Population

Next, let's modify the `get_sh.py` file to wrap the data population statements in a transaction:

```python
import yfinance as yf
from datetime import datetime, timedelta

def yf_getH(cursor, ticker, stock_id):
    """
    Fetch historical stock data for the given ticker and insert it into the database within a transaction.
    
    Parameters:
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    ticker (str): Stock ticker symbol.
    stock_id (int): Unique stock identifier.
    """
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        cursor.connection.start_transaction()
        for date, row in hist.iterrows():
            sql = "INSERT INTO History (HistoryID, Ticker, Date, Price) VALUES (%s, %s, %s, %s)"
            data = (stock_id, ticker, date.strftime('%Y-%m-%d'), row['Close'])
            cursor.execute(sql, data)
            print(f"Inserted history data for {ticker} on {date.strftime('%Y-%m-%d')}")
            stock_id += 1
        cursor.connection.commit()
    except Exception as e:
        cursor.connection.rollback()
        print(f"Error fetching or inserting history for {ticker}: {e}")

def yf_getS(cursor, ticker, stock_id):
    """
    Fetch stock information for the given ticker and insert it into the database within a transaction.
    
    Parameters:
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    ticker (str): Stock ticker symbol.
    stock_id (int): Unique stock identifier.
    """
    try:
        stock = yf.Ticker(ticker)
        stock_info = stock.info
        symbol = stock_info.get('symbol', ticker)
        sector = stock_info.get('sector', 'Unknown')
        price = stock_info.get('regularMarketPreviousClose', 0.0)
        sd = stock_info.get('beta', 0.0)
        eret = round((stock_info.get('forwardEps', 0.0) / price) * 100, 3)
        
        cursor.connection.start_transaction()
        insert_statement = "INSERT INTO Stocks (StockID, Ticker, Sector, Price, SD, ERet) VALUES (%s, %s, %s, %s, %s, %s)"
        data = (stock_id, symbol, sector, price, sd, eret)
        cursor.execute(insert_statement, data)
        cursor.connection.commit()
        print(f"Inserted data for {ticker}")
    except Exception as e:
        cursor.connection.rollback()
        print(f"Error fetching or inserting data for {ticker}: {e}")

def get_stock(connection, cursor, tickers):
    """
    Fetch and store stock data for a list of ticker symbols within a transaction.
    
    Parameters:
    connection (MySQLConnection): MySQL connection object.
    cursor (MySQLCursor): MySQL cursor object to execute SQL queries.
    tickers (list): List of stock ticker symbols.
    """
    stock_id = 1
    try:
        connection.start_transaction()
        for ticker in tickers:
            ticker = ticker.strip()
            yf_getS(cursor, ticker, stock_id)
            yf_getH(cursor, ticker, stock_id)
            stock_id += 1000
        connection.commit()
    except Exception as e:
        connection.rollback()
        print(f"Error fetching or inserting stock data: {e}")
```

### Summary

By wrapping the table creation and data population operations in transactions, we ensure that these operations are atomic, consistent, isolated, and durable. This approach helps maintain the integrity of the database even in the event of errors or failures.

* The `db_setup.py` script now ensures that all table creation statements are executed within a single transaction.
* The `get_sh.py` script ensures that data fetching and insertion operations for each stock are executed within transactions, and the overall process of fetching and storing stock data is also wrapped in a transaction.

Citations:
[1] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/4026eff2-ef75-4a18-b4a3-acd553b8cd14/connect.py
[2] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/6cc1f0df-6f87-428e-b08b-f87acf9ab8f3/portef.py
[3] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/8cace1f0-d268-4480-919d-a71c9d95562c/get_sh.py
[4] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/22098966/ba14aa5e-eaf7-4dcd-a8d4-7c5eddc827eb/db_setup.py
[5] https://www.mysqltutorial.org/mysql-stored-procedure/mysql-transactions/
[6] https://www.tutorialspoint.com/mysql/mysql-transactions.htm
[7] https://dev.mysql.com/doc/refman/8.4/en/atomic-ddl.html
[8] https://www.freecodecamp.org/news/how-to-use-mysql-transactions/